<a href="https://colab.research.google.com/github/bhouldridge/repertoire/blob/main/LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Linear Regression: Project 2

By: Benjamin Houldirdge

July 30th 2020

##Problem Definition

This is an unidimensional regression problem, which predicts the sale price of a property based on available information. The ideal outcome is to provide information about what a property's true market value is. 

##Data Collection

###Import nessasary modules

In [ ]:
import numpy as np
import pandas as pd
import sklearn.model_selection as model_selection
from sklearn import preprocessing
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

###Mount to Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###Import Data file

In [ ]:
# data_path = '/content/drive/My Drive/Exercises - Benjamin/Project 2/Copy of Housing Data.csv'
data_path = '/content/drive/My Drive/Data Science Student Material/Project 2/Data/Housing Data.csv'
data_original = pd.read_csv(data_path)

###Reserve copy of original data

In [ ]:
data = data_original.copy()

##Data Cleaning

###Inspect Data

In [ ]:
data.shape

In [ ]:
data.info()

Check Columns and first few rows of data

In [ ]:
data.head()

###Drop Unessasary Columns

In [ ]:
data = data.drop(['PID'], axis=1);

###Fill Nan

In [ ]:
data['Lot Frontage'] = data['Lot Frontage'].fillna(0)
data['Alley'] = data['Alley'].fillna('None')
data['Mas Vnr Type'] = data['Mas Vnr Type'].fillna('None')
data['Mas Vnr Area'] = data['Mas Vnr Area'].fillna(0)
data['Fireplace Qu'] = data['Fireplace Qu'].fillna('None')
data['Pool QC'] = data['Pool QC'].fillna('None')
data['Fence'] = data['Fence'].fillna('No Fence')
data['Misc Feature'] = data['Misc Feature'].fillna('None')
data['Total Bsmt SF'] = data['Total Bsmt SF'].fillna(0)
data['Bsmt Full Bath'] = data['Bsmt Full Bath'].fillna(0)
data['Bsmt Half Bath'] = data['Bsmt Half Bath'].fillna(0)
data['Bsmt Unf SF'] = data['Bsmt Unf SF'].fillna(0)
data['Bsmt Qual']= data['Bsmt Qual'].fillna('No Basement')
data['Bsmt Cond'] = data['Bsmt Cond'].fillna('No Basement')
data['Bsmt Exposure'] = data['Bsmt Exposure'].fillna('No Basement')
data['BsmtFin Type 1'] = data['BsmtFin Type 1'].fillna('No Basement')
data['BsmtFin SF 1'] = data['BsmtFin SF 1'].fillna('No Basement')
data['BsmtFin Type 2'] = data['BsmtFin Type 2'].fillna('No Basement')
data['BsmtFin SF 2'] = data['BsmtFin SF 2'].fillna('No Basement')
data['Garage Type'] = data['Garage Type'].fillna('No Garage')
data['Garage Cond'] = data['Garage Cond'].fillna('No Garage')
data['Garage Qual'] = data['Garage Qual'].fillna('No Garage')
data['Garage Area'] = data['Garage Area'].fillna(0)
data['Garage Cars'] = data['Garage Cars'].fillna(0)
data['Garage Finish'] = data['Garage Finish'].fillna('No Garage')
data['Garage Type'] = data['Garage Type'].fillna('No Garage')


Change Nan values in 'Garage Yr Blt'. Nan values with no garage will be one-hoted to a new column titled 'No Garage'. Nan values with a garage will be replaced with the year the home was built.

In [ ]:
data.insert(59, 'No Garage', 0)

In [ ]:
missing_values = pd.isna(data)
for idx, series in data.iterrows():
  if missing_values.at[idx, 'Garage Yr Blt'] == True:
    if data.at[idx, 'Garage Area'] == 0:
      data.at[idx, 'No Garage'] = 1
      data.at[idx, 'Garage Yr Blt'] = 0
    else:
      data.at[idx, 'Garage Yr Blt'] = data.at[idx, 'Year Built']

###Check for Incompatable Data

In [ ]:
def incompatable_check(column1, attribute1, column2, attribute2, dataframe = data):
  """ Return index of all rows for which attribute1 and attribute2 are not compatable."""
  
  incompatable = []
  for idx, series in dataframe.iterrows():
    if series[column1] == attribute1 and series[column2] == attribute2:
      continue
    elif series[column1] != attribute1 and series[column2] != attribute2:
      continue
    else:
      incompatable.append(idx)
  return incompatable

The one incompatability that I found was in row 329. Garage type showes Detchd and the Area is 0. 

In [ ]:
incompatable1 = incompatable_check('Garage Area', 0, 'Garage Type', 'No Garage')
print(incompatable1)

Drop line 

In [ ]:
data = data.drop(329, axis=0)

Showing lines 162, 329, 2005 which where droped in class. 

Line 329 is droped for above reason. 

Line 162 dosn't have any inconsistacny I can find.

Line 2005 looks like "BsmtFin Type 2" is Unf with "BsmtFin SF 2" 0 and "Bsmt Unf SF" 226.0. This looks to be inconsitant but the accompianing documentation seems internaly inconsitant. The documentation discribes BsmtFin Type 1 and 2 as "rating of basment finished area." and yet has unfinished as an option of type. If we threw out every entry with and unfinished type that has 0 square feet but shows a positive number for Bsmt Unf SF then we would have to throw out 768 rows. I'm going to assume that during data colection BsmtFin SF 1 and 2 where left at 0 if the type was unfinished.

In [ ]:
data_original.filter(items=[162,329,2005], axis=0)

In [ ]:
check1 = data[data['BsmtFin Type 2']=='Unf']
check2 = check1[check1['Bsmt Unf SF']!=0]
check3 = check2[check2['BsmtFin Type 1']=='Unf']
len(check3)

Abnormal and partial sales are producing outlires in SalePrice. Keep only the rows whith sale condition is normal.

In [ ]:
data = data[data['Sale Condition'] == 'Normal']

###Reindex Dataframe

In [ ]:
data = data.reset_index(drop=True)

Check for any abnormalities.

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.shape

##Exploratory Data Analysis

###Get Correlations


In [ ]:
def build_correlation_list(data_frame=data):
  """Return a list of highest corilated values from the data dataframe."""

  data_correlation  = data_frame.corr()['SalePrice']
  correlation_list = data_correlation.abs().sort_values(ascending=False)
  correlation_list = correlation_list.drop('SalePrice', axis=0)
  return correlation_list

In [ ]:
correlation_list = build_correlation_list()
print(correlation_list)
print(correlation_list.shape)

### Error Functions

In [ ]:
def numeric_sets(data_frame=data):
  """Produce a dataframe of saleprices and a numeric values."""
  
  features_all = data_frame.drop('SalePrice', axis=1)
  saleprice = data_frame['SalePrice']
  features_numeric = features_all.select_dtypes(include=['int64', 'float64'])
  return saleprice, features_numeric


In [ ]:
def mean_of_rsquared(features, itterations=1000):
  """Retun a dictionary with the mean of the r_squared and rms_error of a model ran itterations times"""

  r_squared_values = []
  rms_predicted_error = []
  for idx in range(itterations):
    train_variables, test_variables, train_saleprice, test_saleprice = model_selection.train_test_split(features, saleprice, test_size=.2)
    model = sm.OLS(train_saleprice, train_variables).fit()
    prediction = model.predict(test_variables)
    rmspe = np.sqrt(np.mean(np.square((test_saleprice - prediction)/test_saleprice)))*100

    rms_predicted_error.append(rmspe)
    r_squared_values.append(model.rsquared)

  return np.mean(r_squared_values), np.mean(rms_predicted_error) 

In [ ]:
def itterate_numeric(num_features):
  """Return an array with the precentage error for the model as features are added."""

  r_squared_values = []
  rms_error_values = []
  for idx in range(0,num_features):
    features_reduced = pd.DataFrame(features_numeric[correlation_list.index[:idx+1]].copy())
    features_reduced['Ones'] = 1

    r_squared, rms_error = mean_of_rsquared(features_reduced)
    r_squared_values.append(r_squared)
    rms_error_values.append(rms_error)
    print( str(idx+1) + ' feature compleet')
  return {'r_squared':r_squared_values, 'rms_error':rms_error_values}

In [ ]:
saleprice, features_numeric = numeric_sets()

###Eliminate Unessasary Numerical Features

Produce cross validation errors for numeric features

In [ ]:
numeric_features_error = itterate_numeric(35)

In [ ]:
print(numeric_features_error)

Graph change in R-Squared and RMS values as features are added to the model

In [ ]:
sns.set()
plt.figure(figsize=(10,8))
plt.plot(numeric_features_error['r_squared'])
plt.xlabel('Number of Features')
plt.ylabel('R-Squared')

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(numeric_features_error['rms_error'])
plt.xlabel('Number of Features')
plt.ylabel('Root Mean Squared Percentage Error')

In [ ]:
print(numeric_features_error['rms_error'])

Find freatures that increase the root mean square error when added

In [ ]:
def find_variables_that_increese_error():
  idx_of_negitive_variables = []
  for idx, value in enumerate(numeric_features_error['rms_error']):
    if value - numeric_features_error['rms_error'][idx-1] > .1:
      idx_of_negitive_variables.append(idx)
  idx_of_negitive_variables.remove(0)
  print(idx_of_negitive_variables)

  labels = []
  for idx in idx_of_negitive_variables:
    labels.append(correlation_list.index[idx])
  return labels

In [ ]:
labels = find_variables_that_increese_error()
print(labels)

Drop Features that increse error.

In [ ]:
data = data.drop(columns=labels)

Drop unuseful features

In [ ]:
data = data.drop(columns=correlation_list.index[-6:])

Recalculate corrilation list and graph rms error precentage

In [ ]:
correlation_list = build_correlation_list(data)
print(correlation_list)
print(correlation_list.shape)

In [ ]:
saleprice, features_numeric = numeric_sets()
numeric_features_error = itterate_numeric(len(correlation_list))
plt.figure(figsize=(10,8))
plt.plot(numeric_features_error['rms_error'])
plt.xlabel('Number of Features')
plt.ylabel('Root Mean Squared Percentage Error')

In [ ]:
print(numeric_features_error['rms_error'])

###Seperate Numeric and Object Features

In [ ]:
data_numeric = data.select_dtypes(exclude='object').copy()
data_objects = data.select_dtypes(include='object').copy()

###Scale Numeric Data

In [ ]:
scaler = preprocessing.StandardScaler()
scaled_data = scaler.fit_transform(data_numeric)
scaled_data = pd.DataFrame(scaled_data, columns= data_numeric.columns)
scaled_data['SalePrice'] = data_numeric['SalePrice'] 

###Scaled and original data sets produce the same results.

Train and test original data

In [ ]:
data_numeric_variables = data_numeric.drop('SalePrice', axis=1)
data_numeric_variables['Ones'] = 1
data_numeric_saleprice = data_numeric['SalePrice']
model = sm.OLS(data_numeric_saleprice, data_numeric_variables).fit()
prediction1 = model.predict(data_numeric_variables)

Train and test scaled data[link text](https://)

In [ ]:
scaled_data_variables = scaled_data.drop('SalePrice', axis=1)
scaled_data_variables['Ones'] = 1
scaled_data_saleprice = data_numeric['SalePrice']
model = sm.OLS(scaled_data_saleprice, scaled_data_variables).fit()
prediction2 = model.predict(scaled_data_variables)

Compare predictions of original and scaled data.

In [ ]:
print((prediction1 - prediction2).max())

The largest discrepency is on the order of e-8 so we can safely assume that the scaled results are equivilant for our uses.

In [ ]:
data_numeric_scaled = scaled_data.copy()

###One Hot catigorical Features

Find out which values should be one hotted.

In [ ]:
def rmspe(train, outcomes, itterations=50):
  """calculate the rmspe itteration times and return the mean."""

  error_vector = []
  for idx in range(itterations):
    x_train, x_test, y_train, y_test = model_selection.train_test_split(train, outcomes, test_size=.2)
    model = sm.OLS(y_train, x_train).fit()
    predict = model.predict(x_test)
    error_vector.append(np.sqrt(np.mean(np.square((y_test - predict)/y_test)))*100)
  return np.mean(error_vector)

In [ ]:
def one_hot_efficacy(numeric=data_numeric_scaled, objects=data_objects):
  """One-hot each variable and test it's change on preformence."""

  #set up numeric variables, and calculate error with no one_hot featurs.
  outcomes = numeric['SalePrice']
  numeric = numeric.drop(['SalePrice'], axis=1)
  numeric['Ones'] = 1
  base_error = rmspe(numeric, outcomes)
  errors = []
  

  for column in objects.columns:
    feature = pd.get_dummies(objects[column])
    train_data = pd.concat([numeric, feature], axis=1)
    errors.append(rmspe(train_data, outcomes))
  
  errors_series = pd.Series(data=errors, index=objects.columns)
  change_in_error = errors_series.subtract(base_error)
  change_in_error_sorted = change_in_error.sort_values()
  return change_in_error_sorted


List the object values by change in error when cross validated.

In [ ]:
objects_ranked = one_hot_efficacy()
print(objects_ranked)

Function that returns the error as one-hot features are added.

In [ ]:
def one_hot_efficacy(numeric=data_numeric_scaled, objects=data_objects, ranked_list=objects_ranked):
  """One-hot variables one by one and test the affect on preformence."""

  #set up numeric variables.
  outcomes = numeric['SalePrice']
  numeric = numeric.drop(['SalePrice'], axis=1)
  numeric['Ones'] = 1
  train_data = numeric.copy()
  errors = []

  for column in ranked_list.index:
    feature = pd.get_dummies(objects[column])
    train_data = pd.concat([train_data, feature], axis=1)
    errors.append(rmspe(train_data, outcomes))

  errors_series = pd.Series(data=errors, index=ranked_list.index)
  return errors_series

In [ ]:
one_hot_error = one_hot_efficacy()

Graph the change in error as features are added.

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(range(45), one_hot_error)
plt.xlabel('Number of Features')
plt.ylabel('Root Mean Squared Precentage Error')

From the graph it looks like the low point in error is around 15. Print out the actual values to make sure. 

In [ ]:
one_hot_error

Select only the first 15 features for one hot coding and produce a dataframe with all the

In [ ]:
def one_hot_final(numeric=data_numeric_scaled, objects=data_objects, ranked_list=objects_ranked):
  """One-hot first 15 variables one by one and test the affect on preformence."""

  #set up numeric variables.
  outcomes = numeric['SalePrice']
  numeric['Ones'] = 1
  train_data = numeric.copy()

  for column in ranked_list.index[:15]:
    feature = pd.get_dummies(objects[column])
    train_data = pd.concat([train_data, feature], axis=1)
  
  return train_data

###Create Final Data Sets

In [ ]:
data_final = one_hot_final().copy()
print(data_final.shape)
data_final.head()

In [ ]:
'Ones' in data_final

###Nonlinear Fitting

look as an array of powers to see which works best.

In [ ]:
data_final['SalePrice'].index[data_final['SalePrice'].apply(np.isnan)]

In [ ]:
print(type(data_final))
print(data_final.isna().sum().sum())
print(type(data_final['SalePrice']))
print(data_final['SalePrice'].isna().sum())

In [ ]:
power_array = 1/np.arange(1,10,1)
train = data_final.drop(['SalePrice'], axis=1)
outcomes = data_final['SalePrice']
mean_errors = []

for power in power_array:
  validation_error = []
  for idx in range(500):
    x_train, x_test, y_train, y_test = model_selection.train_test_split(train, outcomes, test_size=0.2)
    y_train_raised_to_power = y_train.pow(power)
    model = sm.OLS(y_train_raised_to_power, x_train).fit()
    predict = model.predict(x_test)
    final_predict = predict.pow(1/power)
    validation_error.append(np.sqrt(np.mean(np.square((y_test - final_predict)/y_test)))*100)

  mean_errors.append(np.mean(validation_error))

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(power_array, mean_errors)
plt.xlabel('Power')
plt.ylabel('Root Mean Squared Precentage Error')

In [ ]:
print(list(zip(mean_errors,power_array)))

From the Graph and the list above we can see that using a power of 1/3 gives us the best outcomes.

##Data Processing

###Split Training Data Set

Split Data

In [ ]:
data_final_features = data_final.drop(['SalePrice'], axis=1).copy()
data_outcome = data_final['SalePrice'].copy()

Apply power to outcomes

In [ ]:
 data_final_outcome = data_outcome.pow(1/3)

###Improt Blind Data

In [ ]:
blind_data_path = '/content/drive/My Drive/Exercises - Benjamin/Project 2/Copy of Housing Data Blind Test.csv'
blind_data = pd.read_csv(blind_data_path)

###Inspect Blind Data

In [ ]:
blind_data.head()

In [ ]:
blind_data.shape

No Nan values

In [ ]:
blind_data.isna().sum().sum()

###Seperate Numerical and Object Data Sets

In [ ]:
blind_data_numerical = blind_data.select_dtypes(exclude='object').copy()
blind_data_objects = blind_data.select_dtypes(include='object').copy()

###One Hot Encode Object Data

In [ ]:
blind_data_onehot = pd.get_dummies(blind_data_objects)
blind_data_onehot.shape

###Scale Numerical Data

In [ ]:
scaler = preprocessing.StandardScaler()
scaled_data = scaler.fit_transform(blind_data_numerical)
blind_data_scaled = pd.DataFrame(scaled_data, columns= blind_data_numerical.columns)
blind_data_scaled.shape

Add a ones column

In [ ]:
blind_data_scaled['Ones'] = 1

###Join One Hot and Scaled Numerical Data

In [ ]:
blind_data_joined = pd.concat([blind_data_scaled, blind_data_onehot], axis=1)
blind_data_joined.shape

create test data with only columns also in the traingin data.

In [ ]:
column_intersection = blind_data_joined.columns.intersection(data_final_features.columns)
blind_data_final = blind_data_joined.loc[:, column_intersection].copy()

###Inspect Final Blind Test Data

In [ ]:
print(blind_data_final.shape)
blind_data_final.head()

###Train model

create training features with only colums also in the blind test data.

In [ ]:
training_data_final = data_final_features.loc[:, blind_data_final.columns] 

In [ ]:
final_model = sm.OLS(data_final_outcome, training_data_final).fit()

###Predict Sale Price Using Blind Data

In [ ]:
test_prediction_blind_data = final_model.predict(blind_data_final).pow(3)
print(test_prediction_blind_data)

###Export Prediction

In [ ]:
test_prediction_blind_data.to_csv('/content/drive/My Drive/Exercises - Benjamin/Project 2/Blind Test Predictions.csv')

##Data Visualization

### Nonlinear Fitting

####Power Used

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(power_array, mean_errors)
plt.xlabel('Power')
plt.ylabel('Root Mean Squared Precentage Error')

####Without Nonlinear Fitting

In [ ]:
model = sm.OLS(data_outcome, training_data_final).fit() 
prediction = model.predict(training_data_final)

plt.figure(figsize=(10,8))
plt.scatter(data_outcome,prediction)
plt.plot([0, max(prediction)],[0, max(prediction)], c='red');

plt.xlabel('Real Sales Price',fontsize=16);
plt.ylabel('Predictions',fontsize=16);

plt.axis('equal');

####With Nonlinear Fitting

In [ ]:
model = sm.OLS(data_final_outcome, training_data_final).fit() 
prediction = model.predict(training_data_final)

prediction = prediction.pow(3)

plt.figure(figsize=(10,8))
plt.scatter(data_outcome,prediction)
plt.plot([0, max(prediction)],[0, max(prediction)], c='red')

plt.xlabel('Real Sales Price',fontsize=16)
plt.ylabel('Predictions',fontsize=16)

plt.axis('equal');

###One Hot Features Added

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(range(45), one_hot_error)
plt.xlabel('Number of Features')
plt.ylabel('Root Mean Squared Precentage Error')

###Numeric Features Added

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(numeric_features_error['rms_error'])
plt.xlabel('Number of Features')
plt.ylabel('Root Mean Squared Percentage Error')